In [1]:
import pandas as pd
import numpy as np 
data = pd.read_csv('training_data.csv')  ##use this data cv F1 0.50  
#data = pd.read_csv('facies_vectors.csv')  ##use this data(two more wells) cv F1 0.48 
#data.describe()
data_test = data[data['Well Name'] == 'SHANKLE']
data = data[data['Well Name'] != 'SHANKLE']
#data2 = data.sample(n=729)
#data2.describe()   # use random sampled 729 events gives 
#F1 scroe 0.51 for CV(nonstratified split), 0.36 for Test 
#F1 scroe 0.55 for CV(stratified split), 0.41 for Test 

#data['BoundaryFlag'] = pd.Series(0,index=data.index)  # add one column. 
#previousface=-1
#for index, row in data.iterrows():
#    if row['Facies']!=previousface: 
#        data.at[index, 'BoundaryFlag'] =  1   
#        data.at[index-1, 'BoundaryFlag'] =  1    
#        data.set_value(index, 'BoundaryFlag',1) ##old version
#        previousface = row['Facies']
#data = data[data['BoundaryFlag']==0]
#data= data.drop(['BoundaryFlag'],axis=1)
#data.describe
#w/o boundary in data, F1 score 0.61 for CV and 0.43 for Test 
#w boundary in data, F1 score 0.50 for CV and 0.55 for Test
data.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3222, 3223, 3224, 3225, 3226, 3227, 3228, 3229, 3230, 3231],
           dtype='int64', length=2783)

In [2]:
features = ['GR', 'ILD_log10', 'DeltaPHI','PHIND','PE','NM_M', 'RELPOS']
feature_vectors = data[features]
facies_labels = data['Facies']

feature_vectors_test = data_test[features]
facies_labels_test = data_test['Facies']

facies_labels.describe()
# Store well labels and depths
well = data['Well Name'].values
depth = data['Depth'].values
well_test = data_test['Well Name'].values
depth_test = data_test['Depth'].values
# Fill 'PE' missing values with mean
from sklearn import preprocessing
imp = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(feature_vectors)
feature_vectors = imp.transform(feature_vectors)
imp.fit(feature_vectors_test)
feature_vectors_test = imp.transform(feature_vectors_test)


In [3]:



# Feature windows concatenation function
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row

    return X_aug


# Feature gradient computation function
def augment_features_gradient(X, depth):
    
    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
        
    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad


# Feature augmentation function
def augment_features(X, well, depth, N_neig=1):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
    return X_aug, padded_rows

In [4]:
X_aug, padded_rows = augment_features(feature_vectors, well, depth)
X_aug_test, padded_rows_test = augment_features(feature_vectors_test, well_test, depth_test)

X_aug.shape

(2783, 28)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_aug)
#from sklearn import preprocessing
#scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_aug)
scaled_features = scaler.transform(X_aug) #ndarray now. 
scaler_test = StandardScaler().fit(X_aug_test)
#scaler_test = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_aug_test)
scaled_features_test = scaler.transform(X_aug_test) #ndarray now. 
#scaled_features.shape
#scaler = StandardScaler().fit(feature_vectors)
#scaled_features = scaler.transform(feature_vectors) #ndarray now. 

#df3 = pd.DataFrame(np.random.randn(10, 5),columns=['a', 'b', 'c', 'd', 'e'])
#df3.describe
scaled_features.shape

(2783, 28)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.05, random_state=42,stratify=facies_labels)
#X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.05, random_state=42)
#y_train.value_counts()
X_train.shape

(2643, 28)

In [7]:
from sklearn.metrics import classification_report
target_names = ['SS', 'CSiS', 'FSiS', 'SiSh','MS', 'WS', 'D','PS', 'BS']

In [8]:
from xgboost.sklearn import  XGBClassifier
f1_results = np.zeros(100)
for seed in range(100):
    np.random.seed(seed)
    clf = XGBClassifier(learning_rate=0.12,max_depth=3,min_child_weight=10,n_estimators=150,seed=seed,colsample_bytree=0.9)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(scaled_features_test)
    #print('random seed = ',seed)
    rpt_string = classification_report(facies_labels_test, y_pred,target_names=target_names)
    f1_results[seed]=(float(rpt_string[len(rpt_string)-15:len(rpt_string)-5]))
print(f1_results)
print(np.average(f1_results))
print(np.std(f1_results))


C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\Homeuser\Anaconda3\lib\site-packages\sk

C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\Homeuser\Anaconda3\lib\site-packages\sk

C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\Homeuser\Anaconda3\lib\site-packages\sk

C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\Homeuser\Anaconda3\lib\site-packages\sk

C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\Homeuser\Anaconda3\lib\site-packages\sk

C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\Homeuser\Anaconda3\lib\site-packages\sk

[0.6  0.59 0.6  0.58 0.61 0.59 0.6  0.59 0.59 0.6  0.58 0.59 0.6  0.6
 0.6  0.6  0.6  0.61 0.59 0.6  0.59 0.6  0.58 0.59 0.59 0.6  0.59 0.61
 0.59 0.59 0.59 0.59 0.58 0.59 0.58 0.59 0.59 0.58 0.59 0.59 0.6  0.59
 0.59 0.6  0.59 0.6  0.6  0.59 0.59 0.6  0.59 0.59 0.57 0.6  0.59 0.59
 0.59 0.6  0.6  0.58 0.59 0.59 0.58 0.58 0.59 0.6  0.6  0.59 0.58 0.59
 0.59 0.6  0.58 0.6  0.59 0.6  0.58 0.6  0.6  0.6  0.6  0.6  0.59 0.59
 0.58 0.59 0.59 0.6  0.6  0.6  0.58 0.59 0.57 0.6  0.58 0.59 0.59 0.59
 0.59 0.58]
0.5920000000000001


C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))


In [9]:
#y_cv_pred = clf.predict(X_cv) 
#print(classification_report(y_cv, y_cv_pred,target_names=target_names))


In [11]:
print(np.std(f1_results))

0.008124038404635967
